# Importing the libraries

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import os
from langchain_huggingface import HuggingFaceEmbeddings

# Loading and testing the Question Answering models

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

print("Starting download and caching of Hugging Face models...")

QA_MODELS = [
      "deepset/tinyroberta-squad2",
      "deepset/roberta-base-squad2",
      "google-bert/bert-large-uncased-whole-word-masking-finetuned-squad"
]

In [ ]:
for model_name in QA_MODELS:
    try:
        # print(f"Loading model: {model_name}")
        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        # This is the essential part for downloading and caching the model
        model = AutoModelForQuestionAnswering.from_pretrained(model_name)
        
        # The following lines simulate a quick inference to ensure everything is loaded correctly
        question = "How many programming languages does BLOOM support?"
        context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."
        inputs = tokenizer(question, context, return_tensors="pt")

        with torch.no_grad():
            outputs = model(**inputs)

        answer_start_index = outputs.start_logits.argmax()
        answer_end_index = outputs.end_logits.argmax()
        predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
        decoded_answer = tokenizer.decode(predict_answer_tokens)
        
        print(f"Successfully loaded and tested: {model_name}. Predicted answer: '{decoded_answer}'")

    except Exception as e:
        print(f"Failed to load model {model_name}. Error: {e}")

print("Model loading process complete.")

# Loading and Testing the Embedding Models

In [ ]:
EMBEDDING_MODEL_NAMES = ["all-MiniLM-L6-v2", 
              "all-mpnet-base-v2",
              "all-MiniLM-L12-v2"]

for model_name in EMBEDDING_MODEL_NAMES:
    model_name = 'sentence-transformers/' + model_name
    embedding_model = HuggingFaceEmbeddings(model_name=model_name)
    print(embedding_model)
    docs = ['this is a test', 'I want to see if the model works properly', 'from the docker container.']
    a = embedding_model.embed_documents(docs)
    print(a)